# **Image Classification in PyTorch**

Image classification is a central task in computer vision. Building better classifiers to classify what object is present in a picture is an active area of research, as it has applications stretching from autonomous vehicles to medical imaging.

Such models are perfect to use with Gradio's image input component. In this notebook, we have a web demo to classify images using Gradio. We can build the whole web application in Python.

In [6]:
# Install gradio silently (no output shown)

import subprocess

# Run pip install and capture the output
result = subprocess.run(
    ["pip", "install", "-q", "gradio"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# Check if it succeeded and print the last line of stdout
if result.returncode == 0:
    print("✔️", result.stdout.strip().split('\n')[-1])  # Only print last line
else:
    print("❌ Installation failed:")
    print(result.stderr)

✔️    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00


**Setting up the image classification model**

We will need an image classification model. On this Notebook, we will use a pretrained Resnet-18 model, as it is easily downloadable from PyTorch Hub.

In [8]:
import torch
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True).eval()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


**Defining a predict function**

Next, we will need to define a function that takes in the user input, which in this case is an image, and returns the prediction. The prediction should be returned as a dictionary whose keys are class name and values are confidence probabilities. We will load the class names from this text file.

In [3]:
import requests

from PIL import Image
from torchvision import transforms

# Download human-readable labels for ImageNet.
response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")

def predict(inp):
 inp = transforms.ToTensor()(inp).unsqueeze(0)
 with torch.no_grad():
  prediction = torch.nn.functional.softmax(model(inp)[0], dim=0)
  confidences = {labels[i]: float(prediction[i]) for i in range(1000)}
 return confidences

**Creating a Gradio interface**

Now that we have our predictive function set up, we can create a Gradio Interface around it.

In this case, the input component is a drag-and-drop image component. To create this input, we use Image(type=“pil”) which creates the component and handles the preprocessing to convert that to a PIL image.

The output component will be a Label, which displays the top labels in a nice form. Since we don't want to show all 1,000 class labels, we will customize it to show only the top 3 images by constructing it as Label(num_top_classes=3).

Finally, we'll add one more parameter, the examples, which allows us to prepopulate our interfaces with a few predefined examples.

In [9]:
import gradio as gr

gr.Interface(fn=predict,
       inputs=gr.Image(type="pil"),
       outputs=gr.Label(num_top_classes=3),
       examples=["/content/lion.jpg", "/content/cheetah.jpg"]).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://38cbf3704cc4518894.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 🚀 Demo

Check out the live app: [Image Classification](https://38cbf3704cc4518894.gradio.live/)